In [1]:
from PIL import PILLOW_VERSION

/home/abhishek/anaconda3/envs/ml_project/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in a future release. Use __version__ instead.
  """Entry point for launching an IPython kernel.


In [2]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
code_folder = os.getcwd()
annotation_folder = os.path.join(code_folder,'../../dataset/Stanford/Annotation')
images_folder = os.path.join(code_folder,'../../dataset/Stanford/Images')

In [4]:
transform = transforms.Compose([transforms.Resize((255,255)), transforms.ToTensor()])
dataset = datasets.ImageFolder(images_folder, transform=transform)

In [5]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [8]:
with open(os.path.join(code_folder, '../../dataset/label.txt'), 'w') as f:
  for images, labels in dataloader:
    f.write(str(images))
    print(images[0].shape)

torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
torch.Size([3, 255, 255])
